In [1]:
import numpy as np
import csv
import cv2
import sklearn

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, MaxPooling2D, Dropout
from keras.layers.convolutional import Convolution2D

# Constants
data_path = "data_old/"
image_path = data_path + "IMG/"
left_image_angle_correction = 0.20
right_image_angle_correction = -0.20
csv_data = []
processed_csv_data = []

Using TensorFlow backend.


In [2]:
# Method to pre-process the input image
def pre_process_image(image):
    # Since cv2 reads the image in BGR format and the simulator will send the image in RGB format
    # Hence changing the image color space from BGR to RGB
    colored_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Cropping the image
    #cropped_image = colored_image[60:140, :]
    # Downscaling the cropped image
    #resized_image = cv2.resize(cropped_image, None, fx=0.25, fy=0.4, interpolation=cv2.INTER_CUBIC)
    return colored_image #cropped_image


In [3]:
# Reading the content of csv file
with open(data_path + 'driving_log.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    # Skipping the headers
    next(csv_reader, None)
    for each_line in csv_reader:
        csv_data.append(each_line)

# Getting shape of processed image
first_img_path = image_path + csv_data[0][0].split('/')[-1]
first_image = cv2.imread(first_img_path)
processed_image_shape = pre_process_image(first_image).shape

print(processed_image_shape)

(160, 320, 3)


In [4]:
images = []
measurements = []

for Line in csv_data:
    current_image_path = image_path + Line[0].split('/')[-1]
    current_image = cv2.imread(current_image_path)
    current_image = pre_process_image(current_image)
    images.append(current_image)

    measurement = float(Line[3])
    measurements.append(measurement)
  #------------------------------------------------
    current_image_path = image_path + Line[1].split('/')[-1]
    current_image = cv2.imread(current_image_path)
    current_image = pre_process_image(current_image)
    images.append(current_image)

    measurement = float(Line[3]) + 0.2
    measurements.append(measurement)
  #-------------------------------------------------
    current_image_path = image_path + Line[2].split('/')[-1]
    current_image = cv2.imread(current_image_path)
    current_image = pre_process_image(current_image)
    images.append(current_image)

    measurement = float(Line[3]) - 0.2
    measurements.append(measurement)

augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)


X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

#X_train = np.array(images)
#y_train = np.array(measurements)


In [5]:
# My final model architecture
from keras.layers.convolutional import Cropping2D

model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=processed_image_shape))
model.add(Cropping2D(cropping=((50,20), (0,0))))

model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation="relu"))
#model.add(MaxPooling2D())
#model.add(Dropout(0.25))

model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation="relu"))
#model.add(MaxPooling2D())

model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation="relu"))
#model.add(MaxPooling2D())

model.add(Convolution2D(64, 3, 3, activation="relu"))
#model.add(MaxPooling2D())

model.add(Convolution2D(64, 3, 3, activation="relu"))
#model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(100))  #100
#model.add(Dropout(0.20))
model.add(Dense(50))   # 50
model.add(Dense(10))   # 10
model.add(Dense(1))


In [6]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch = 3)

model.save('model.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/3
38572/38572 [==============================] - 1290s - loss: 0.0185 - val_loss: 0.0198
Epoch 2/3
38572/38572 [==============================] - 1131s - loss: 0.0139 - val_loss: 0.0189
Epoch 3/3
38572/38572 [==============================] - 905s - loss: 0.0121 - val_loss: 0.0186


In [7]:
from keras.models import load_model

def loadModel(modelPath):
    """
    Loads the model `modelPath`.
    """
    model = load_model(modelPath)
    return model


In [8]:
loadModel('model.h5').summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 90, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 43, 158, 24)   1824        cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 20, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________

In [9]:
loadModel('model_LeNet_5.h5').summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 80, 320, 3)    0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 76, 316, 6)    456         lambda_1[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 38, 158, 6)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 34, 154, 6)    906         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

# dict_keys(['loss', 'val_loss'])
# Loss
# [0.045153103061875123, 0.031848681302726264, 0.027354457760211899]
# Validation Loss
# [0.033896364609334075, 0.028193514087644336, 0.026007735237962643]
loss = [0.045153103061875123, 0.031848681302726264, 0.027354457760211899]
valid_loss = [0.033896364609334075, 0.028193514087644336, 0.026007735237962643]

### plot the training and validation loss for each epoch
plt.plot(loss)
plt.plot(valid_loss)
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.grid(color='black', linestyle='--', linewidth=1)
plt.show()